## FullAdder 

This notebook walks through the implementation of a full adder circuit.

Start by importing `Magma`.

In [1]:
import magma as m

Define a Python function that constructs a full adder. A full adder has three single bit inputs, and returns the sum and the carry. The sum is the *exclusive or* of the 3 bits, the carry is 1 if any two of the inputs bits are 1.

![Full Adder](images/full_adder_logisim.png)

The values passed to `fulladder` are `Magma` values. 
These values are of type `Bit`.
The python bitwise operators are overloaded to compute logical functions of the `Magma` values (this corresponds to constructing the circuits to compute these logical functions).

We compute the two desired output values, and return them as a Python tuple.

In [2]:
def fulladder(A, B, C):
    return A^B^C, A&B|B&C|C&A

The the IceStick board has two headers `J1` and `J3`, 
each connected to 8 GPIO pins on the ICE40 FPGA. 
The type of `J1` and `J3` is `Bits(8)`. `Bits(n)` is a length `n` array of `Bit` values.
Arrays of bits can be accessed using the standard Python indexing notation (e.g. `[-1]`).

In these tutorials, `J1` will be used for inputs and `J3` will be used for outputs.
We configure three pins in `J1` as inputs,
and two pins in `J3` as outputs.
We also turn on each pin that we are using.

In [3]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.J1[0].input().on()
icestick.J1[1].input().on()
icestick.J1[2].input().on()
icestick.J3[0].output().on()
icestick.J3[1].output().on()

import mantle lattice ice40
import mantle lattice mantle40


J3[1] = GPIO(name="PIO2_16")

Now setup the top level `main` program that runs on the ICE40. The arguments to the main program are the headers `J1` and `J3`. We call `fulladder` with three single bit inputs from `J1`. We then `wire` the sum and carry outputs returned by `fulladder` to `J3`.

Note that to connect the outputs of the full adder to `J3` we need to call `wire`. 
Assigning a Python variable 
(such as assigning the values returned by `fulladder` to `s` and `c`)
is not the same as wiring it.
`Magma` values are Python objects,
and so assigning an object to a variable creates a reference to that value.

In [4]:
main = icestick.DefineMain()

_sum, cout = fulladder(main.J1[0], main.J1[1], main.J1[2])
m.wire(_sum, main.J3[0])
m.wire(cout, main.J3[1])

m.EndDefine()

Now we can use the magma's `compile` function to generate verilog code targeting a board from the `"lattice"` vendor (this tells magma the collateral it needs to generate to program a vendor specific board).

In [5]:
m.compile('build/fulladder', main, vendor="lattice")

compiling XOr2
compiling And2
compiling Or2
compiling main


Then we can use `yosys` and the `icestorm` tools to compile and program the FPGA.

In [6]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif fulladder.blif' fulladder.v
arachne-pnr -q -d 1k -o fulladder.txt -p fulladder.pcf fulladder.blif 
icepack fulladder.txt fulladder.bin
iceprog fulladder.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


You can test the program by connecting up some switches and LEDs to the headers.  To see which physical pins correspond to the J3 and J1 pins we used, we can inspect the `pcf` file.

In [7]:
with open("build/fulladder.pcf", "r") as full_adder_pcf:
    print(full_adder_pcf.read())

set_io J1[0] 112
set_io J1[1] 113
set_io J1[2] 114
set_io J3[1] 61
set_io J3[0] 62



Here is an example circuit to see the sum of the inputs displayed on a set of LEDs.
![full adder test circuit on breadboard1](images/full_adder_1.jpg)
![full adder test circuit on breadboard2](images/full_adder_2.jpg)
![full adder test circuit on breadboard3](images/full_adder_3.jpg)

## Verilog

If you know verilog, it is instructive to look at the verilog code that was generated.

In [8]:
with open("build/fulladder.v", "r") as full_adder_verilog:
    print(full_adder_verilog.read())

module XOr2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h6666)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module And2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h8888)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module Or2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'hEEEE)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module main (input [2:0] J1, output [1:0] J3);
wire  inst0_O;
wire  inst1_O;
wire  inst2_O;
wire  inst3_O;
wire  inst4_O;
wire  inst5_O;
wire  inst6_O;
XOr2 inst0 (.I({J1[1],J1[0]}), .O(inst0_O));
XOr2 inst1 (.I({J1[2],inst0_O}), .O(inst1_O));
And2 inst2 (.I({J1[1],J1[0]}), .O(inst2_O));
And2 inst3 (.I({J1[2],J1[1]}), .O(inst3_O));
Or2 inst4 (.I({inst3_O,inst2_O}), .O(inst4_O));
And2 inst5 (.I({J1[0],J1[2]}), .O(inst5_O));
Or2 inst6 (.I({inst5_O,inst4

The logical functions are implemented using verilog modules `And2`, `Or2`, and `XOr2`. These in turn are implemented using 4-bit LUTs using the ICE40 primitive module `SB_LUT4`. The top level `main` module instances the logical functions and wires them up. 
It is all quite simple.